1) Multi-regression output problem --> predict two labels: longitude and latitude

2) Re-write problem as multi-class problem through clustering
To define the classes, the last points per trip from the training data will be divided into clusters. When we have a large enough number of clusters, the clusters should be small enough to have their points close to their centroids. 

K-Means is a good approach as it minimizes the sum of squared distances within the cluster. Also it scales well to a large number of clusters. 

So the approach then predicts: **Probability of final destination of a trip being located in a specific cluster**

Steps:
- Get optimal number of clusters with help of interia
- Generate clusters with K-Means for each trip in training data
- Predict cluster for each trip in testing data
- Get centroids for each cluster 
- Calculate distance from points in cluster to centroid to evaluate

In [1]:
import os
import sys
import pandas as pd
import numpy as np
import datetime
from datetime import datetime
import time
import json
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import awswrangler as wr
import random
from sklearn.cluster import KMeans
import dask.array as da
from dask_ml import cluster
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import geojson

In [2]:
is_sagemaker_notebook = False

In [3]:
if is_sagemaker_notebook:
    prefix = "/home/ec2-user/SageMaker"
else:
    prefix = "/Users/Q619505/PycharmProjects/personal-projects"

utils_path = os.path.join(f'{prefix}/ml-project-taxi-prediction/src/utils/')
pp_path = os.path.join(f'{prefix}/ml-project-taxi-prediction/src/preprocessing')

if utils_path not in sys.path:
    sys.path.append(utils_path)

if pp_path not in sys.path:
    sys.path.append(pp_path)

In [4]:
from utils import *
from geo_spatial import convert_polyline_to_geojson_format, extract_lat_lon

ImportError: cannot import name 'extract_lat_lon' from 'geo_spatial' (/Users/Q619505/PycharmProjects/personal-projects/ml-project-taxi-prediction/src/preprocessing/geo_spatial.py)

In [ ]:
test_data = pd.read_csv(f'{prefix}/ml-project-taxi-prediction/data/processed/test_data.csv')
train_data = pd.read_csv(f'{prefix}/ml-project-taxi-prediction/data/processed/train_data.csv')

In [ ]:
train_data = convert_polyline_to_geojson_format(data=train_data, name_column='dest_point')
test_data = convert_polyline_to_geojson_format(data=test_data, name_column='dest_point')

In [ ]:
train_data = extract_lat_lon(train_data,'start_point')
test_data = extract_lat_lon(test_data,'start_point')

train_data = extract_lat_lon(train_data,'dest_point')
test_data = extract_lat_lon(test_data,'dest_point')

In [ ]:
X = np.vstack(train_data.dest_point)
# transform destintion point to dask format
X_da = da.from_array(X)

- Inertia/sum of squared distance shows still significant reduction for 4000 clusters, therefore choose n_clusters = 4000
- Due to high execution time more iterations are not considered

In [ ]:
#inertia_per_k = [(k, KMeans(n_clusters=k, init='k-means++', verbose=True, random_state=1).fit(X_da).inertia_)
#  for k in range(1000,5000,1000)]

One iteration is sufficient as single points need to be clustered, the nature of the data already provides the clusters

In [ ]:
random.seed(5)

In [ ]:
pipe = Pipeline([('scaler', StandardScaler()),
                 ('clustering', KMeans(n_clusters = 4000,
                                       init ='k-means++',
                                       n_init = 1, 
                                       verbose=True,
                                       random_state=1
                                      )
                 )])

In [ ]:
cluster_label = pipe.fit_predict(X_da)

In [ ]:
train_data['cluster_label']=cluster_label.reshape(-1,1)

In [ ]:
centers_scaled = pipe[1].cluster_centers_

In [ ]:
centers = pipe[0].inverse_transform(centers_scaled)

In [ ]:
centers = pd.DataFrame(centers, columns=['CENTER_LON','CENTER_LAT'])

In [ ]:
centers = centers.reset_index()

In [ ]:
sns.set()
plt.hist(train_data.cluster_label)
plt.show()

In [ ]:
X_test = np.vstack(test_data.dest_point)
X_da_test = da.from_array(np.array(X_test))
cluster_label = pipe.predict(X_da_test)
test_data['cluster_label']=cluster_label.reshape(-1,1)

In [ ]:
sns.set()
plt.hist(test_data.cluster_label)
plt.show()

In [ ]:
assert(centers.index.max() == train_data.cluster_label.max())

In [ ]:
test_data = pd.merge(test_data, centers[['index','center_lon','center_lat']], 
                           how='left', 
                           left_on='cluster_label',
                           right_on='index')
train_data = pd.merge(train_data, centers[['index','center_lon','center_lat']], 
                           how='left', 
                           left_on='cluster_label',
                           right_on='index')

In [ ]:
n_cluster = train_data.cluster_label.nunique()

In [ ]:
if is_sagemaker_notebook:
    aw.s3.to_parquet(df=train_data, path=f's3://think-tank-casestudy/preprocessed_data/n_cluster_{n_cluster}/train_data_clustered.parquet', dataset=True, partition_cols=['taxi_id'])
    aw.s3.to_parquet(df=test_data, path=f's3://think-tank-casestudy/preprocessed_data/n_cluster_{n_cluster}/test_data_clustered.parquet', dataset=True, partition_cols=['taxi_id'])

else:
    train_data.to_csv(f'{prefix}/ml-project-taxi-prediction/data/processed/train_data_clustered.csv', header=True, index=False)
    test_data.to_csv(f'{prefix}/ml-project-taxi-prediction/data/processed/test_data_clustered.csv', header=True, index=False)